<a href="https://colab.research.google.com/github/sarabert96/Colexification/blob/main/04_BootstrapProbabilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bootstrap and calculating of probabilities
The file does a bootstrapping and for every cicle it calculates the probability of finding a specific colexification in any of the families. 

These probabilies are saved in a list and subsequently in a file.

In [107]:
import pandas as pd
import numpy as np
from collections import Counter
import random

Import cleaned version of colexification dataframe from Drive

In [108]:
!gdown --id 1YmX2hOvKM5vVRiT7UlCnDKezbYTRZ-kG #df_colexifications

Downloading...
From: https://drive.google.com/uc?id=1YmX2hOvKM5vVRiT7UlCnDKezbYTRZ-kG
To: /content/df_colexifications.csv
7.57MB [00:00, 66.4MB/s]


In [109]:
df_colex = pd.read_csv('df_colexifications.csv')

Take the number of families

In [110]:
nFam = len(df_colex.groupby('Family').size())

Add concepticon_pair column

In [111]:
df_colex['Concepticon_pair']=list(zip(df_colex['Concepticon_Gloss.x'],df_colex['Concepticon_Gloss.y']))

In [112]:
df_colex.head(2)

,clics_form,Concepticon_ID.x,Glottocode,Concepticon_Gloss.x,Family,variety,Concepticon_ID.y,Concepticon_Gloss.y,Concepticon_pair
0,s@,1369,hrus1242,gold,Hruso,Hruso Aka Jamiri,1927,bamboo,"(gold, bamboo)"
1,s@,1369,hrus1242,gold,Hruso,Hruso Aka Jamiri,946,blood,"(gold, blood)"


In [113]:
# df_fam --> df with families and varieties
df_fam = df_colex[['Family','Glottocode']].copy()

In [114]:
df_fam=df_fam.drop_duplicates()

In [115]:
dicFam = df_fam.groupby(['Family']).size().to_dict() # dictionary with number of variety

### BOOTSTRAP function

In [116]:
from statistics import mean
import time
times = []

def timer(start,end):
  hours, rem = divmod(end-start, 3600)
  minutes, seconds = divmod(rem, 60)
  tempo='TIME: '+"{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)
  f.write(tempo + '\n')
  print(tempo)
  if times: # if not empty
    times.append(end-start-times[-1])
  else:
    times.append(end-start)
     


def colex_prob_any(x, suppDf):
  # x = df of only one colex_pair
  sumProb = sum(x['Prob_colex_fam'])
  prob = sumProb / nFam # prob of finding colex in any fam, ex. if 1 always found
  newEl = [x['Colex_pair'].iloc[0],x['Colex_pair'].iloc[0][0],x['Colex_pair'].iloc[0][1], prob]
  suppDf.append(newEl)


def concept_glot_rep(x, suppDf):
  # x = df divided by concepticon inside only one family
  totGlot = len(x['Glottocode']) # how many languages have that colexification faking to have different families from bootstrap --> NO SET
  # NB we don't have repetitions of Concepticon_pair in a variety, if we have it, it's just because we did the bootstrap with replacement
  newEl = [x['Family'].iloc[0], x['Concepticon_pair'].iloc[0], totGlot]
  suppDf.append(newEl)


def colex_fam(x, suppDf):
  # x = df divided by family
  x.groupby('Concepticon_pair').apply(lambda y: concept_glot_rep(y, suppDf))


def bootstrapping(cicle):
  # cicle = number of repetitions
  print ("Running bootstrapping function")
  start_time = time.time()

  listBoot = []
  
  for k in range (cicle):
    # create new df with random varieties
    new_df_fam = df_fam.groupby(['Family']).apply(lambda x: x.sample(n=(len(x['Glottocode'])), replace=True)).reset_index(drop = True)
    # create list with varieties chosen
    listVar=new_df_fam.Glottocode.tolist()
    print("List of linguistic varieties chosen")
    # new df
    boot_df = pd.DataFrame()

    for v in listVar:
      # for every variety, take all colexification and put them into boot_df
      boot_df=boot_df.append(df_colex.loc[df_colex['Glottocode'] == v], ignore_index = True)

    suppDf = [] # new empty list

    print("Taking families grouped by concepticon pair and counting number of varieties with the same colexification")

    boot_df.groupby(['Family']).apply(lambda x: colex_fam(x, suppDf))
    print("Created dataframe with colexification from chosen varieties")
    # new df from list obtained by functions
    df_sum = pd.DataFrame(suppDf, columns=['Family', 'Colex_pair', 'Glot_sum'])

    df_sum['Prob_colex_fam']='' # new empty column

    for i in range(len(df_sum['Colex_pair'])):
      fam = df_sum['Family'].iloc[i] # take the family for the row
      rep = df_sum['Glot_sum'].iloc[i] # take glottocode repetition
      div = rep / dicFam[fam]
      df_sum['Prob_colex_fam'].iloc[i] = div # rate of colex within a fam --> prob of finiding that colex inside fam
    suppDf = [] # empty the list

    print("Counting colex frequency among all families")
    df_sum.groupby(['Colex_pair']).apply(lambda x: colex_prob_any(x, suppDf))

    listBoot.append(suppDf) # add list to list of bootstrap
    # df_colex_prob = pd.DataFrame(suppDf, columns=['Colex_pair', 'Prob']) # to save in df
    print("Finished cicle number", k)
    f.write("Finished cicle number " + str(k) + '\n')
    timer(start_time, time.time())
    print('\n')

  return listBoot



In [117]:
pd.options.mode.chained_assignment = None  # default='warn'
# avoid to show warnings

In [118]:
f = open("statistics_time.txt", "w")

random.seed(70) # random number 
lBoot = bootstrapping(100) # number is how many cicles

hours, rem = divmod(mean(times), 3600)
minutes, seconds = divmod(rem, 60)
#print('Average time of each cicle: : ',"{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
f.write('Average time of each cicle: : '+"{:0>2}:{:0>2}:{:05.2f} \n".format(int(hours),int(minutes),seconds))
f.close()

Running bootstrapping function
List of linguistic varieties chosen
Taking families grouped by concepticon pair and counting number of varieties with the same colexification
Created dataframe with colexification from chosen varieties
Counting colex frequency among all families
Finished cicle number 0
TIME: 00:01:38.01


List of linguistic varieties chosen
Taking families grouped by concepticon pair and counting number of varieties with the same colexification
Created dataframe with colexification from chosen varieties
Counting colex frequency among all families
Finished cicle number 1
TIME: 00:03:12.55


List of linguistic varieties chosen
Taking families grouped by concepticon pair and counting number of varieties with the same colexification
Created dataframe with colexification from chosen varieties
Counting colex frequency among all families
Finished cicle number 2
TIME: 00:04:51.32


List of linguistic varieties chosen
Taking families grouped by concepticon pair and counting number

In [119]:
import pickle
with open("listBoot.txt", "wb") as fp:   #Pickling
  pickle.dump(lBoot, fp)

In [120]:
from google.colab import files
files.download('listBoot.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>